# DeepStream 활용하여 YOLO 모델 실행하기

이 강의에서는 YOLOv11 모델을 TensorRT 엔진 파일(`yolo11n.engine`)로 실행하여 DeepStream 애플리케이션에 통합하는 과정을 다룹니다. DeepStream은 NVIDIA의 고성능 비디오 분석 SDK로, TensorRT와 함께 동작하여 효율적인 딥러닝 추론을 제공합니다.

## **1. DeepStream과 TensorRT 개요**
- **DeepStream**: NVIDIA의 스트림 기반 비디오 분석 SDK로, TensorRT 엔진 파일을 활용하여 실시간 추론을 지원합니다.
- **TensorRT 엔진 파일**: ONNX 모델에서 변환된 최적화된 파일로, DeepStream에서 추론에 사용됩니다.

DeepStream 애플리케이션의 기본 구성:
1. 소스 입력 (RTSP, 파일 등)
2. 추론 수행 (YOLOv11 엔진 활용)
3. 결과 시각화 또는 저장

## **2. 환경 설정**
DeepStream 애플리케이션을 실행하기 전, 필요한 설정을 진행합니다.

### **필요 파일**
1. TensorRT 엔진 파일: `yolo11n.engine`
2. DeepStream 구성 파일: `config_infer_primary_yolo11n.txt`

구성 파일은 모델 경로 및 추론 파라미터를 정의합니다.

### **구성 파일 예제**
아래는 `config_infer_primary_yolo11n.txt`의 예제입니다:

```txt
[property]
gpu-id=0
model-engine-file=yolo11n.engine
labelfile-path=labels.txt
batch-size=1
network-mode=0
num-detected-classes=80
interval=0
gie-unique-id=1
```

## **3. DeepStream 애플리케이션 코드**
이제 Python을 사용하여 DeepStream 애플리케이션을 실행하는 코드를 작성합니다. 이 코드는 TensorRT 엔진 파일을 로드하고, 비디오 스트림에서 추론을 수행합니다.

In [ ]:
# 필요한 라이브러리 임포트
import gi
import sys

# GStreamer와 DeepStream 플러그인을 임포트합니다.
gi.require_version('Gst', '1.0')
from gi.repository import Gst

# GStreamer 초기화
Gst.init(None)

In [ ]:
# DeepStream 파이프라인 생성
pipeline = Gst.parse_launch(
    "filesrc location=sample_video.mp4 ! decodebin ! nvstreammux name=mux batch-size=1 ! "
    "nvinfer config-file-path=config_infer_primary_yolo11n.txt ! nvtracker ! "
    "nvmultistreamtiler ! nvvideoconvert ! nvdsosd ! nveglglessink"
)

# 파이프라인 확인
if not pipeline:
    print("파이프라인 생성 실패")
    sys.exit(1)

In [ ]:
# 파이프라인 실행
def run_pipeline():
    try:
        pipeline.set_state(Gst.State.PLAYING)
        print("DeepStream 파이프라인 실행 중...")

        # GStreamer 버스에서 메시지 수신
        bus = pipeline.get_bus()
        while True:
            msg = bus.timed_pop_filtered(Gst.CLOCK_TIME_NONE, Gst.MessageType.ERROR | Gst.MessageType.EOS)
            if msg:
                t = msg.type
                if t == Gst.MessageType.ERROR:
                    err, debug = msg.parse_error()
                    print(f"에러: {err}, 디버그 정보: {debug}")
                    break
                elif t == Gst.MessageType.EOS:
                    print("End-Of-Stream 도달")
                    break
    except Exception as e:
        print(f"파이프라인 실행 중 오류 발생: {e}")
    finally:
        pipeline.set_state(Gst.State.NULL)

# 실행
run_pipeline()

## **4. 실행 방법**
1. 위 코드와 `config_infer_primary_yolo11n.txt` 파일을 동일한 디렉토리에 저장합니다.
2. `yolo11n.engine` 파일도 같은 위치에 배치합니다.
3. Python 파일을 실행하여 DeepStream 파이프라인을 실행합니다.

```bash
python3 deepstream_yolo11n.py
```

## **5. 요약 및 정리**
- TensorRT 엔진 파일과 DeepStream 구성 파일을 활용하여 실시간 추론을 수행하는 방법을 학습했습니다.
- YOLOv11 모델을 사용하여 비디오 스트림에서 객체 탐지를 수행했습니다.
추가적으로 원하는 모델로 확장하거나 커스터마이징이 가능합니다.